In [233]:
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

plt.ion()

In [234]:
x = torch.linspace(-math.pi, math.pi, 2000).unsqueeze(-1).pow(torch.tensor([1, 2, 3]))
y = torch.sin(x)

In [235]:
x

tensor([[ -3.1416,   9.8696, -31.0063],
        [ -3.1384,   9.8499, -30.9133],
        [ -3.1353,   9.8301, -30.8205],
        ...,
        [  3.1353,   9.8301,  30.8205],
        [  3.1384,   9.8499,  30.9133],
        [  3.1416,   9.8696,  31.0063]])

In [236]:
y

tensor([[ 8.7423e-08, -4.3030e-01,  3.9829e-01],
        [-3.1430e-03, -4.1240e-01,  4.8172e-01],
        [-6.2863e-03, -3.9436e-01,  5.6085e-01],
        ...,
        [ 6.2863e-03, -3.9436e-01, -5.6085e-01],
        [ 3.1430e-03, -4.1240e-01, -4.8172e-01],
        [-8.7423e-08, -4.3030e-01, -3.9829e-01]])

In [237]:
# approximate y = sin x using RL

n_states = 3
n_actions = 2

In [238]:
model = torch.nn.Sequential(
    torch.nn.Linear(n_states, 10),
    torch.nn.Linear(10, n_actions)
)

In [239]:
model(x)

tensor([[-15.9177,   0.0847],
        [-15.8685,   0.0830],
        [-15.8194,   0.0814],
        ...,
        [ 16.0546,  -2.4048],
        [ 16.1044,  -2.4104],
        [ 16.1542,  -2.4159]], grad_fn=<AddmmBackward0>)

In [240]:
class Model(nn.Module):
    def __init__(self, n_states, n_actions):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(n_states, 10)
        self.layer2 = nn.Linear(10, n_actions)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        return self.layer2(x)

In [241]:
model = Model(n_states, n_actions)

In [242]:
model(x)

tensor([[ 3.0785,  2.3064],
        [ 3.0687,  2.2973],
        [ 3.0590,  2.2882],
        ...,
        [ 3.7602, -4.9552],
        [ 3.7697, -4.9701],
        [ 3.7792, -4.9850]], grad_fn=<AddmmBackward0>)

In [243]:
state = torch.randn((3), dtype=torch.float)

In [244]:
state

tensor([ 0.3534, -1.0440,  1.8523])

In [245]:
model(state)

tensor([-0.2242,  0.0301], grad_fn=<ViewBackward0>)

In [246]:
x

tensor([[ -3.1416,   9.8696, -31.0063],
        [ -3.1384,   9.8499, -30.9133],
        [ -3.1353,   9.8301, -30.8205],
        ...,
        [  3.1353,   9.8301,  30.8205],
        [  3.1384,   9.8499,  30.9133],
        [  3.1416,   9.8696,  31.0063]])

In [247]:
model(x)

tensor([[ 3.0785,  2.3064],
        [ 3.0687,  2.2973],
        [ 3.0590,  2.2882],
        ...,
        [ 3.7602, -4.9552],
        [ 3.7697, -4.9701],
        [ 3.7792, -4.9850]], grad_fn=<AddmmBackward0>)

In [248]:
state = torch.tensor([[-3.1, 9.8, -30.0]])

In [249]:
action = model(state).max(1).indices.view(1,1)

In [250]:
model(state)

tensor([[2.9729, 2.1846]], grad_fn=<AddmmBackward0>)

In [251]:
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state'))

In [252]:
class Memory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)
    def push(self, *args):
        self.memory.append(Transition(*args))
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    def __len__(self):
        return len(self.memory)

In [253]:
# what happens when you choose the max action? You want to go to another state. Which state is it?

In [254]:
#get step functon that returns next_state and reward
#increase or decrease y-value of y = sin x by 0.01

def step(state, action):
    if action == 1:
        next_state = state + 0.01
    else:
        next_state = state - 0.01
    reward = torch.tensor([1], dtype=torch.long)
    return next_state, reward

In [255]:
class Agent:
    def __init__(self, n_states, n_actions):
        self.batch_size = 128
        self.gamma = 0.7
        self.epsilon = 0.99
        self.decay = 0.01
        self.lr = 1e-4
        
        self.model = Model(n_states, n_actions)
        self.memory = Memory(1000)
        self.optimizer = optim.AdamW(self.model.parameters(), lr=self.lr, amsgrad=True)
    
    def get_action(self, state):
        sample = random.random()
        epsilon = self.epsilon*self.decay
        if sample > epsilon:
            with torch.no_grad():
                return self.model(state).max(1).indices.view(1, 1)
        else:
            random_action = random.choice([0, 1])
            return torch.tensor([[random_action]], dtype=torch.long)
        
    def update(self):
        if len(self.memory) < self.batch_size:
            return
        transitions = self.memory.sample(self.batch_size)
        batch = Transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        next_state_batch = torch.cat(batch.next_state)
        
        q_values = self.model(state_batch).gather(1, action_batch)
        with torch.no_grad():
            next_state_values = self.model(next_state).max(1).values
        q_pred = (next_state_values * self.gamma) + reward_batch
        
        criterion = nn.SmoothL1Loss()
        loss = criterion(q_values, q_pred.unsqueeze(1))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [256]:
agent = Agent(n_states, n_actions)

episodes = []
state = torch.tensor([[-3.1, 9.8, -30.0]])

for i in range(10000):
    print(state, action)
    action = agent.get_action(state)
    next_state, reward = step(state, action)
    agent.memory.push(state, action, reward, next_state)
    state = next_state
    agent.update()

tensor([[ -3.1000,   9.8000, -30.0000]]) tensor([[0]])
tensor([[ -3.0900,   9.8100, -29.9900]]) tensor([[1]])
tensor([[ -3.0800,   9.8200, -29.9800]]) tensor([[1]])
tensor([[ -3.0700,   9.8300, -29.9700]]) tensor([[1]])
tensor([[ -3.0600,   9.8400, -29.9600]]) tensor([[1]])
tensor([[ -3.0500,   9.8500, -29.9500]]) tensor([[1]])
tensor([[ -3.0400,   9.8600, -29.9400]]) tensor([[1]])
tensor([[ -3.0300,   9.8700, -29.9300]]) tensor([[1]])
tensor([[ -3.0200,   9.8800, -29.9200]]) tensor([[1]])
tensor([[ -3.0100,   9.8900, -29.9100]]) tensor([[1]])
tensor([[ -3.0000,   9.9000, -29.9000]]) tensor([[1]])
tensor([[ -2.9900,   9.9100, -29.8900]]) tensor([[1]])
tensor([[ -2.9800,   9.9200, -29.8800]]) tensor([[1]])
tensor([[ -2.9700,   9.9300, -29.8700]]) tensor([[1]])
tensor([[ -2.9600,   9.9400, -29.8600]]) tensor([[1]])
tensor([[ -2.9500,   9.9500, -29.8500]]) tensor([[1]])
tensor([[ -2.9400,   9.9600, -29.8400]]) tensor([[1]])
tensor([[ -2.9300,   9.9700, -29.8300]]) tensor([[1]])
tensor([[ 

tensor([[ -0.1200,  12.7801, -27.0199]]) tensor([[1]])
tensor([[ -0.1100,  12.7901, -27.0099]]) tensor([[1]])
tensor([[ -0.1000,  12.8001, -26.9999]]) tensor([[1]])
tensor([[ -0.0900,  12.8101, -26.9899]]) tensor([[1]])
tensor([[ -0.0800,  12.8201, -26.9799]]) tensor([[1]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[1]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[1]])
tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[1]])
tensor([[ -0.0400,  12.8601, -26.9399]]) tensor([[1]])
tensor([[ -0.0300,  12.8701, -26.9299]]) tensor([[1]])
tensor([[-2.0003e-02,  1.2880e+01, -2.6920e+01]]) tensor([[1]])
tensor([[-1.0003e-02,  1.2890e+01, -2.6910e+01]]) tensor([[1]])
tensor([[-2.5779e-06,  1.2900e+01, -2.6900e+01]]) tensor([[1]])
tensor([[ 9.9974e-03,  1.2910e+01, -2.6890e+01]]) tensor([[1]])
tensor([[ 1.9997e-02,  1.2920e+01, -2.6880e+01]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0500,  

tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[1]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[0]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[1]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[0]])
tensor([[ -0.0800,  12.8201, -26.9799]]) tensor([[0]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[1]])
tensor([[ -0.0800,  12.8201, -26.9799]]) tensor([[0]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[1]])
tensor([[ -0.0800,  12.8201, -26.9799]]) tensor([[0]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[1]])
tensor([[ -0.0800,  12.8201, -26.9799]]) tensor([[0]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[1]])
tensor([[ -0.0800,  12.8201, -26.9799]]) tensor([[0]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[1]])
tensor([[ -0.0800,  12.8201, -26.9799]]) tensor([[0]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[1]])
tensor([[ -0.0800,  12.8201, -26.9799]]) tensor([[0]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[1]])
tensor([[ 

tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[1]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[0]])
tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[1]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[0]])
tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[1]])
tensor([[ -0.0400,  12.8601, -26.9399]]) tensor([[1]])
tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[0]])
tensor([[ -0.0400,  12.8601, -26.9399]]) tensor([[1]])
tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[0]])
tensor([[ -0.0400,  12.8601, -26.9399]]) tensor([[1]])
tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[0]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[0]])
tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[1]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[0]])
tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[1]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[0]])
tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[1]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[0]])
tensor([[ 

tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[1]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[0]])
tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[1]])
tensor([[ -0.0400,  12.8601, -26.9399]]) tensor([[1]])
tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[0]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[0]])
tensor([[ -0.0500,  12.8501, -26.9499]]) tensor([[1]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[0]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[0]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[1]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[0]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[1]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[0]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[1]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[0]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[1]])
tensor([[ -0.0700,  12.8301, -26.9699]]) tensor([[0]])
tensor([[ -0.0600,  12.8401, -26.9599]]) tensor([[1]])
tensor([[ 

...

tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[ 

tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[ 

tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[ 

tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0300,  12.9301, -26.8699]]) tensor([[0]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[1]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[ 

tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[  0.0400,  12.9401, -26.8599]]) tensor([[0]])
tensor([[  0.0500,  12.9501, -26.8499]]) tensor([[1]])
tensor([[ 

In [257]:
state

tensor([[  0.0400,  12.9401, -26.8599]])

In [258]:
next_state

tensor([[  0.0400,  12.9401, -26.8599]])

In [259]:
agent.memory.sample(10)

[Transition(state=tensor([[  0.0400,  12.9401, -26.8599]]), action=tensor([[1]]), reward=tensor([1]), next_state=tensor([[  0.0500,  12.9501, -26.8499]])),
 Transition(state=tensor([[  0.0400,  12.9401, -26.8599]]), action=tensor([[0]]), reward=tensor([1]), next_state=tensor([[  0.0300,  12.9301, -26.8699]])),
 Transition(state=tensor([[  0.0500,  12.9501, -26.8499]]), action=tensor([[0]]), reward=tensor([1]), next_state=tensor([[  0.0400,  12.9401, -26.8599]])),
 Transition(state=tensor([[  0.0400,  12.9401, -26.8599]]), action=tensor([[1]]), reward=tensor([1]), next_state=tensor([[  0.0500,  12.9501, -26.8499]])),
 Transition(state=tensor([[  0.0300,  12.9301, -26.8699]]), action=tensor([[1]]), reward=tensor([1]), next_state=tensor([[  0.0400,  12.9401, -26.8599]])),
 Transition(state=tensor([[  0.0300,  12.9301, -26.8699]]), action=tensor([[1]]), reward=tensor([1]), next_state=tensor([[  0.0400,  12.9401, -26.8599]])),
 Transition(state=tensor([[  0.0400,  12.9401, -26.8599]]), acti

In [260]:
agent.model.state_dict()

OrderedDict([('layer1.weight',
              tensor([[ 0.4603,  0.2414,  0.0532],
                      [ 0.3470, -0.1358,  0.3514],
                      [ 0.4877,  0.1701,  0.2304],
                      [ 0.4232, -0.5003,  0.0347],
                      [ 0.0433, -0.0388, -0.5632],
                      [-0.5008,  0.2492,  0.5037],
                      [ 0.0708,  0.5781,  0.0012],
                      [-0.5634, -0.4293, -0.1911],
                      [-0.3567, -0.3288,  0.5290],
                      [-0.5270, -0.4806, -0.3230]])),
             ('layer1.bias',
              tensor([ 0.4052, -0.4257,  0.1061,  0.4698,  0.2077,  0.2760, -0.5019, -0.5212,
                       0.0446, -0.3290])),
             ('layer2.weight',
              tensor([[ 0.3142,  0.2672, -0.1135, -0.0760,  0.0540, -0.0102,  0.2128, -0.0325,
                       -0.2757,  0.1064],
                      [-0.1856,  0.1036,  0.2971, -0.2835,  0.0932, -0.2273,  0.3178,  0.0246,
                       -0.2

In [ ]:
# verify results
# plot results